In [1]:
#!pip pandas openpyxl selenium webdriver-manager

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
dados = pd.read_excel("Excel/UBA_Cadastradas_SAI.xlsx")
dados_ativos = dados[dados['Status'] == 'Ativa'].copy()
dados_ativos['Empresas'] = dados_ativos['Nome fantasia'] + " / " + dados_ativos['Razão social'] + " / " + dados_ativos['Município'] + " / " + dados_ativos['UF']
dados_ativos.to_excel("Excel/Empresas_Ativas.xlsx")

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
links_maps = [] 

for row_number in range(len(dados_ativos['Empresas'])):
    nome_empresa = dados_ativos['Empresas'].iloc[row_number]
    url_busca = f"https://www.google.com/maps/search/{nome_empresa.replace(' ', '+')}"
    driver.get(url_busca)
    time.sleep(3)
    url_final = driver.current_url
    # print(url_final)
    links_maps.append(url_final)

driver.quit()
dados_ativos["URL maps"] = links_maps
dados_ativos.to_excel("Excel/EMPRESAS_ATIVAS_COM_LINK.xlsx")
print("terminou")

KeyboardInterrupt: 

In [ ]:
latitudes = []
longitudes = []

for url in dados_ativos["URL maps"]:
    match = re.search(r'@(-?\d+\.\d+),(-?\d+\.\d+)', url)
    if match:
        latitudes.append(float(match.group(1)))
        longitudes.append(float(match.group(2)))
    else:
        latitudes.append(None)
        longitudes.append(None)

dados_ativos["Latitude"] = latitudes
dados_ativos["Longitude"] = longitudes
dados_ativos.to_excel("Excel/EMPRESAS_ATIVAS_COM_COORDENADAS.xlsx")
print("terminou")

In [ ]:
# So de sacanagem pra ver isso
contagem_por_uf = dados_ativos['UF'].value_counts().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
contagem_por_uf.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title("Quantidade de Algodoeiras Ativas por UF")
plt.xlabel("UF")
plt.ylabel("Quantidade de Algodoeiras")
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=1)

plt.show()